<h1 id="Use-Keras-Pretrained-Models-dataset">1. Use Keras Pretrained Models dataset<a class="anchor-link" href="#Use-Keras-Pretrained-Models-dataset" target="_self">¶</a></h1><p>Kernels can't use network connection to download pretrained keras model weights.
This dataset helps you to apply your favorite pretrained model in the Kaggle Kernel environment. 
You can find more details <a href="https://www.kaggle.com/gaborfodor/keras-pretrained-models" target="_top">here</a>.</p>
<p>We have to copy the pretrained models to the cache directory (~/.keras/models) where keras is looking for them.</p>

In [ ]:
import fnmatch
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
import gc
from IPython.display import SVG
import cv2
np.random.seed(21)

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns

print(os.listdir('../input/plant-seedlings-classification/train/'))

In [ ]:
path = '../input/plant-seedlings-classification/train/'
train_label = []
train_img = []
label2num = {'Loose Silky-bent':0, 'Charlock':1, 'Sugar beet':2, 'Small-flowered Cranesbill':3,
             'Common Chickweed':4, 'Common wheat':5, 'Maize':6, 'Cleavers':7, 'Scentless Mayweed':8,
             'Fat Hen':9, 'Black-grass':10, 'Shepherds Purse':11}
for i in tqdm(os.listdir(path)):
    label_number = label2num[i]
    new_path = path+i+'/'
    for j in fnmatch.filter(os.listdir(new_path), '*.png'):
        temp_img = image.load_img(new_path+j, target_size=(200,200))
        train_label.append(label_number)
        temp_img = image.img_to_array(temp_img)
        train_img.append(temp_img)

train_img = np.array(train_img)

train_y=pd.get_dummies(train_label)
train_y = np.array(train_y)
train_img=preprocess_input(train_img)

print('Training data shape: ', train_img.shape)
print('Training labels shape: ', train_y.shape)
gc.collect()

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)
gc.collect()

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
EPOCHS = 20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_img,train_y, test_size=0.2, random_state = 42)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = np.array(y_train, dtype="float32")
y_test = np.array(y_test, dtype="float32")

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
gc.collect()

In [ ]:
# def convert(image, label):
#     image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
#     return image, label

# def flip_aug(image,label):
#     image,label = convert(image, label)
#     image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
#     image = tf.image.flip_left_right(image)
#     return image,label

# def rotate_aug(image,label):
#     image,label = convert(image, label)
#     image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
#     image = tf.image.rot90(image)
#     return image,label

# def crop_aug(image,label):
#     image,label = convert(image, label)
#     image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
#     image = tf.image.central_crop(image, central_fraction=0.8)
#     return image,label


# def pad_light_aug(image,label):
#     image,label = convert(image, label)
#     image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
#     image = tf.image.resize_with_crop_or_pad(image, 220, 220) # Add 6 pixels of padding
#     image = tf.image.random_crop(image, size=[220, 220, 3]) # Random crop back to 224x224x3
#     image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
#     return image,label

In [ ]:
# train_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((X_train, y_train))
#     .repeat()
#     .map(flip_aug, num_parallel_calls=AUTO)
#     .map(rotate_aug, num_parallel_calls=AUTO)
#     .map(crop_aug, num_parallel_calls=AUTO)
#     .map(pad_light_aug, num_parallel_calls=AUTO)
#     .batch(BATCH_SIZE)
#     .prefetch(AUTO)
# )

# valid_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((X_test, y_test))
#     .batch(BATCH_SIZE)
#     .cache()
#     .prefetch(AUTO)
# )
# gc.collect()

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def vgg16_model(num_classes=None):

    model = VGG16(weights='imagenet', include_top=False,input_shape=(200,200,3))
    model.trainable = False
    
    x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)
    
    return model

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fscore(y_true, y_pred):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f_score = 2 * (p * r) / (p + r + K.epsilon())
    return f_score

In [ ]:
from keras import backend as K
with strategy.scope():
    num_classes=12
    model = vgg16_model(num_classes)
    optimizer = tf.optimizers.Adam(lr=3e-5 * strategy.num_replicas_in_sync)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',fscore])

model.summary()

In [ ]:
def train_input_fn(batch_size=1024):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.cache() # Loads the data into memory since its such a small dataset
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat() 
    dataset = dataset.batch(batch_size, drop_remainder=True)


    # Return the dataset.
    return dataset

def test_input_fn(batch_size=1024):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)


    # Return the dataset.
    return dataset

In [ ]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

model.fit(train_input_fn,
          batch_size=128,epochs=20, 
          verbose=1, shuffle=True, 
          validation_data=(test_input_fn), callbacks=[model_checkpoint])

In [ ]:
# #Split training data into rain set and validation set
# from sklearn.model_selection import train_test_split
# X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.1, random_state=42)

# #Data augmentation
# '''from keras.preprocessing.image import ImageDataGenerator
# gen_train = ImageDataGenerator( 
#     rotation_range=30,
#     width_shift_range=0.2,
#    height_shift_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True

# )
# gen_train.fit(X_train)

# #Train model
# from keras.callbacks import ModelCheckpoint
# epochs = 10
# batch_size = 32
# model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# model.fit_generator(gen_train.flow(X_train, Y_train, batch_size=batch_size, shuffle=True), 
#                     steps_per_epoch=(X_train.shape[0]//(4*batch_size)), 
#                     epochs=epochs, 
#                     validation_data=(X_valid,Y_valid),
#                     callbacks=[model_checkpoint],verbose=1)
# '''
# from keras.callbacks import ModelCheckpoint
# epochs = 10
# batch_size = 32
# model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# model.fit(X_train,Y_train,
#           batch_size=128,
#           epochs=20,
#           verbose=1, shuffle=True, validation_data=(X_valid,Y_valid), callbacks=[model_checkpoint])

In [ ]:
import matplotlib.pyplot as plt
def plot_model(model):
    plots = [i for i in model.history.history.keys() if i.find('val_') == -1]
    plt.figure(figsize=(10,10))

    for i, p in enumerate(plots):
        plt.subplot(len(plots), 2, i + 1)
        plt.title(p)
        plt.plot(model.history.history[p], label=p)
        plt.plot(model.history.history['val_'+p], label='val_'+p)
        plt.legend()

    plt.show()
    
plot_model(model)

In [ ]:
model.load_weights('weights.h5')

In [ ]:
prob=[]
num=[]
test_img=[]
test_path = '../input/plant-seedlings-classification/test/'
test_all = fnmatch.filter(os.listdir(test_path), '*.png')

test_img=[]
for i in range(len(test_all)):
    path=test_path+'/'+test_all[i]
    temp_img=image.load_img(path,target_size=(200,200))
    temp_img=image.img_to_array(temp_img)
    test_img.append(temp_img) 
test_img=np.array(test_img)    
test_img=preprocess_input(test_img)


test_labels=[]
pred=model.predict(test_img)
num2label =  {0:'Loose Silky-bent', 1:'Charlock',2: 'Sugar beet',3: 'Small-flowered Cranesbill',
              4:'Common Chickweed',5: 'Common wheat',6: 'Maize', 7:'Cleavers', 8:'Scentless Mayweed',
             9: 'Fat Hen', 10:'Black-grass', 11:'Shepherds Purse'}
for i in range(len(test_all)):
    max_score =0
    lab=-1
    for j in range(12):
        if pred[i][j]>max_score:
            max_score=pred[i][j]
            lab=j
    test_labels.append(num2label[lab])


d = {'file': test_all, 'species': test_labels}
df = pd.DataFrame(data=d)
print(df.head(50))

In [ ]:
#Convert dataframe to csv
# df.to_csv("submit.csv",index=False)